In [ ]:

import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso


# Define the time intervals and corresponding MET values
intervals = {
    'walk_1': {'times': ('15:36:00', '15:41:00'), 'MET': 3.0},
    'walk_2': {'times': ('15:43:00', '15:47:00'), 'MET': 3.5},
    'walk_3': {'times': ('15:49:00', '15:53:00'), 'MET': 4.0},
    'walk_4': {'times': ('15:54:30', '15:56:00'), 'MET': 4.5},
    'run_1': {'times': ('15:59:00', '16:03:00'), 'MET': 8.0},
    'run_2': {'times': ('16:06:30', '16:09:30'), 'MET': 10.0}
}


# Convert time strings to datetime objects
for key in intervals:
    start_time_str, end_time_str = intervals[key]['times']
    intervals[key]['times'] = (
        datetime.strptime(start_time_str, '%H:%M:%S'),
        datetime.strptime(end_time_str, '%H:%M:%S')
    )

# Function to extract features from acceleration data
def detrended_fluctuation_analysis(signal):
    n = len(signal)
    lags = np.arange(2, n // 2)
    tau = np.zeros(lags.size)
    
    for i, lag in enumerate(lags):
        segments = n // lag
        reshaped_signal = signal[:segments * lag].reshape(segments, lag)
        local_trend = np.polyval(np.polyfit(np.arange(lag), reshaped_signal.T, 1), np.arange(lag))
        detrended = reshaped_signal - local_trend.T
        tau[i] = np.sqrt(np.mean(detrended ** 2))
    
    alpha = np.polyfit(np.log(lags), np.log(tau), 1)[0]
    return alpha

def extract_features(acc_data):
    mean = np.mean(acc_data, axis=0)
    std_dev = np.std(acc_data, axis=0)
    var = np.var(acc_data, axis=0)
    skewness = skew(acc_data, axis=0)
    kurt = kurtosis(acc_data, axis=0)
    quantiles = np.percentile(acc_data, np.arange(10, 110, 10), axis=0)
    alpha = detrended_fluctuation_analysis(acc_data[:, 3])  # Using magnitude for DFA
    return [mean, std_dev, var, skewness, kurt, quantiles, alpha]
    # return [mean, std_dev, var, skewness, kurt]

# Define high-pass filter parameters
def butter_highpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Read the data from the CSV file and process it
acc_data = []
with open("/Users/sh/Documents/FdS/Master 2 IDS/S1/Santé AP/Data/data_motet.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        try:
            if len(row) < 4:
                raise ValueError("Row has missing values")
            timestamp = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S.%f')
            x = float(row[1])
            y = float(row[2])
            z = float(row[3])
            acc_data.append([timestamp, x, y, z])
        except (ValueError, IndexError) as e:
            print(f"Skipping row due to error: {e}")
            continue

# Sampling frequency and cutoff frequency
fs = 50.0    # Sampling frequency in Hz
cutoff = 0.3  # Cutoff frequency in Hz

# Process the data for each interval
stats_list = []
for label, info in intervals.items():
    start_time = info['times'][0].time()
    end_time = info['times'][1].time()
    
    interval_data = [row for row in acc_data if start_time <= row[0].time() <= end_time]
    if not interval_data:
        print(f"No data found for interval: {label}")
        continue
    
    interval_data = np.array(interval_data, dtype=object)
    interval_data = interval_data[:, 1:].astype(float)
    x_accel = interval_data[:, 0]
    y_accel = interval_data[:, 1]
    z_accel = interval_data[:, 2]
    
    # Calculate magnitude
    accel_magnitude = np.sqrt(np.square(x_accel) + np.square(y_accel) + np.square(z_accel))
    
    # Apply high-pass filter for gravity compensation
    x_accel_compensated = highpass_filter(x_accel, cutoff, fs)
    y_accel_compensated = highpass_filter(y_accel, cutoff, fs)
    z_accel_compensated = highpass_filter(z_accel, cutoff, fs)
    accel_magnitude_compensated = highpass_filter(accel_magnitude, cutoff, fs)
    
    # Extract features
    features = extract_features(np.column_stack((x_accel_compensated, y_accel_compensated, z_accel_compensated,accel_magnitude_compensated)))
    
    stats_list.append({
        'Interval': label,
        'Mean': features[0],
        'Standard Deviation': features[1],
        'Variance': features[2],
        'Skewness': features[3],
        'Kurtosis': features[4],
        'Quantiles': features[5],
        'DFA': features[6],
        'MET': info['MET']
    })

# Prepare the dataset for regression
X = []
y = []

for stats in stats_list:
    features = [
        stats['Mean'],
        stats['Standard Deviation'],
        stats['Variance'],
        stats['Skewness'],
        stats['Kurtosis']
    ]
    X.append(np.concatenate(features).ravel())
    y.append(stats['MET'])

# Convert lists to numpy arrays for modeling
X = np.array(X)
y = np.array(y)

# Split the data into training and validation sets (e.g., 80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the validation data
y_pred = model.predict(X_val)

# Evaluate the model's performance
mse = mean_squared_error(y_val, y_pred)
print("Mean Squared Error:", mse)

# Predict MET values based on extracted features
predicted_MET = model.predict(X)

# Display actual vs predicted MET values
print("Interval\tActual MET\tPredicted MET")
for idx, stats in enumerate(stats_list):
    print(f"{stats['Interval']}\t{stats['MET']}\t{predicted_MET[idx]}")
plt.show()

# Function to plot intervals
def plot_intervals(acc_data, intervals):
    fig, axs = plt.subplots(len(intervals), 1, figsize=(10, 15))
    
    for i, (label, info) in enumerate(intervals.items()):
        start_time = info['times'][0].time()
        end_time = info['times'][1].time()
        
        interval_data = [row for row in acc_data if start_time <= row[0].time() <= end_time]
        if not interval_data:
            print(f"No data found for interval: {label}")
            continue
        
        interval_data = np.array(interval_data, dtype=object)
        interval_data = interval_data[:, 1:].astype(float)
        x_accel = interval_data[:, 0]
        y_accel = interval_data[:, 1]
        z_accel = interval_data[:, 2]
        
        # Calculate magnitude
        accel_magnitude = np.sqrt(np.square(x_accel) + np.square(y_accel) + np.square(z_accel))
        
        axs[i].plot(accel_magnitude)
        axs[i].set_title(f'Interval {label}')
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Magnitude')
    
    plt.tight_layout()
    plt.show()

# Plot actual vs predicted MET values
plt.plot(y, label='Actual MET')
plt.plot(predicted_MET, label='Predicted MET')
plt.legend()
plt.show()

# Call the plot_intervals function
plot_intervals(acc_data, intervals)

# Plot the features for each interval
features = ['Mean', 'Standard Deviation', 'Variance', 'Skewness', 'Kurtosis']
for feature in features:
    plt.figure(figsize=(10, 6))
    for stats in stats_list:
        plt.plot(stats[feature], label=stats['Interval'])
    plt.title(f'{feature} for each interval')
    plt.xlabel('Feature Index')
    plt.ylabel(feature)
    plt.legend()




plt.show()


# Train a Ridge regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Predict on the validation data using Ridge regression
y_pred_ridge = ridge_model.predict(X_val)

# Evaluate the Ridge model's performance
mse_ridge = mean_squared_error(y_val, y_pred_ridge)
print("Ridge Regression Mean Squared Error:", mse_ridge)

# Train a Lasso regression model
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

# Predict on the validation data using Lasso regression
y_pred_lasso = lasso_model.predict(X_val)

# Evaluate the Lasso model's performance
mse_lasso = mean_squared_error(y_val, y_pred_lasso)
print("Lasso Regression Mean Squared Error:", mse_lasso)


from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=5) 
tree_model.fit(X_train, y_train)



from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
forest_model.fit(X_train, y_train)


from sklearn.ensemble import GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbr_model.fit(X_train, y_train)

import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_model.fit(X_train, y_train)


from sklearn.neural_network import MLPRegressor

mlp_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)


from sklearn.metrics import mean_absolute_error, mean_squared_error

y_val_pred = forest_model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)

print("MSE:", mse)
print("MAE:", mae)

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# CNN
window_size = 50  
stride = 25     

X = []
y = []

for stats in stats_list:
    interval_data = [row for row in acc_data if stats['Interval'] == row[0]]
    data_length = len(interval_data)

    for start in range(0, data_length - window_size + 1, stride):
        window = interval_data[start:start + window_size]
        features = np.array(window).T  # Transformer en (temps, caractéristiques)
        X.append(features)
        y.append(stats['MET'])

X = np.array(X)
y = np.array(y)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train / np.max(X_train)
X_val = X_val / np.max(X_val)


model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(window_size, X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
y_pred_cnn = model.predict(X_val)
mse_cnn = mean_squared_error(y_val, y_pred_cnn)
print("CNN Mean Squared Error:", mse_cnn)
